In [1]:
import os
import pandas as pd

In [2]:
data_dir = "/dataset/SV_sets/voices/Development_Data/Speaker_Recognition/sid_dev/"

### Dev Enroll List

In [23]:
dev_enroll_list = pd.read_csv("/dataset/SV_sets/voices/original_data/sid_dev_lists_and_keys/dev-enroll.lst", 
                              names=["id", "wav"], delimiter=' ')
dev_enroll_list['file'] = dev_enroll_list.wav.apply(lambda x: "/".join(x.split("/")[1:]).rstrip(".wav"))
recs =  dev_enroll_list.id.apply(lambda x: x.split('-'))
info = list(zip(*recs))[3:]
info_df = pd.DataFrame(info).T
info_df.columns = ["room", "noise", "spk_id", "ch_id", "sg_id", "mc_id", "mc_t", "mc_l", "mc_deg"]
dev_enroll_df = pd.concat([dev_enroll_list, info_df], axis=1)
dev_enroll_df.drop(columns="wav", inplace=True)
dev_enroll_df['set'] = "enroll"

In [24]:
dev_enroll_df.head()

,id,file,room,noise,spk_id,ch_id,sg_id,mc_id,mc_t,mc_l,mc_deg,set
0,Lab41-SRI-VOiCES-rm1-none-sp3446-ch144019-sg00...,sp3446/Lab41-SRI-VOiCES-rm1-none-sp3446-ch1440...,rm1,none,sp3446,ch144019,sg0006,mc03,stu,mid,dg080,enroll
1,Lab41-SRI-VOiCES-rm1-none-sp3446-ch144019-sg00...,sp3446/Lab41-SRI-VOiCES-rm1-none-sp3446-ch1440...,rm1,none,sp3446,ch144019,sg0006,mc01,stu,clo,dg080,enroll
2,Lab41-SRI-VOiCES-rm1-none-sp2093-ch143262-sg00...,sp2093/Lab41-SRI-VOiCES-rm1-none-sp2093-ch1432...,rm1,none,sp2093,ch143262,sg0015,mc01,stu,clo,dg080,enroll
3,Lab41-SRI-VOiCES-rm1-none-sp2093-ch143262-sg00...,sp2093/Lab41-SRI-VOiCES-rm1-none-sp2093-ch1432...,rm1,none,sp2093,ch143262,sg0015,mc03,stu,mid,dg080,enroll
4,Lab41-SRI-VOiCES-rm1-none-sp2269-ch088761-sg00...,sp2269/Lab41-SRI-VOiCES-rm1-none-sp2269-ch0887...,rm1,none,sp2269,ch088761,sg0004,mc03,stu,mid,dg100,enroll


### Dev Test List 

In [107]:
dev_test_list = pd.read_csv("/dataset/SV_sets/voices/original_data/sid_dev_lists_and_keys/dev-test.lst", names=["id"])
dev_test_list['file'] = dev_test_list.id.apply(lambda x: "/".join(x.split("/")[1:]).rstrip(".wav"))
recs = dev_test_list.id.apply(lambda x: x.rstrip(".wav").split('-'))
info = list(zip(*recs))[3:]
info_df = pd.DataFrame(info).T
info_df.columns = ["room", "noise", "spk_id", "ch_id", "sg_id", "mc_id", "mc_t", "mc_l", "mc_deg"]
dev_test_df = pd.concat([dev_test_list, info_df], axis=1)
# dev_test_df.drop(columns="wav", inplace=True)
dev_test_df['set'] = "test"

In [122]:
dev_test_df.head()

,id,file,room,noise,spk_id,ch_id,sg_id,mc_id,mc_t,mc_l,mc_deg,set
0,sid_dev/sp3521/Lab41-SRI-VOiCES-rm2-musi-sp352...,sp3521/Lab41-SRI-VOiCES-rm2-musi-sp3521-ch0127...,rm2,musi,sp3521,ch012715,sg0017,mc12,lav,wal,dg090,test
1,sid_dev/sp3521/Lab41-SRI-VOiCES-rm2-musi-sp352...,sp3521/Lab41-SRI-VOiCES-rm2-musi-sp3521-ch0127...,rm2,musi,sp3521,ch012715,sg0006,mc04,lav,mid,dg120,test
2,sid_dev/sp3521/Lab41-SRI-VOiCES-rm2-musi-sp352...,sp3521/Lab41-SRI-VOiCES-rm2-musi-sp3521-ch0127...,rm2,musi,sp3521,ch012715,sg0006,mc09,lav,tbo,dg120,test
3,sid_dev/sp3521/Lab41-SRI-VOiCES-rm2-musi-sp352...,sp3521/Lab41-SRI-VOiCES-rm2-musi-sp3521-ch1759...,rm2,musi,sp3521,ch175962,sg0025,mc06,lav,far,dg060,test
4,sid_dev/sp3521/Lab41-SRI-VOiCES-rm2-musi-sp352...,sp3521/Lab41-SRI-VOiCES-rm2-musi-sp3521-ch0127...,rm2,musi,sp3521,ch012715,sg0017,mc11,lav,ceo,dg090,test


In [123]:
from IPython.display import Audio

In [124]:
Audio("/dataset/SV_sets/voices/sid_dev/sp0032/Lab41-SRI-VOiCES-rm2-babb-sp0032-ch021625-sg00

In [127]:
Audio("/dataset/SV_sets/voices/sid_dev/sp0032/Lab41-SRI-VOiCES-rm2-babb-sp0032-ch021631-sg00

### Total Dev List

In [110]:
dev_df = pd.concat([dev_enroll_df, dev_test_df], sort=False)
dev_df.to_csv("/dataset/SV_sets/voices/voices_dev.csv", index=False)

In [111]:
wavs = dev_df.file.apply(lambda x: "sid_dev/"+x+".wav")
ids = dev_df.id
pd.DataFrame(ids).assign(wav=wavs).to_csv("/dataset/SV_sets/voices/kaldi_files/wav.scp", index=False, sep=' ', header=None)
spk2utt = dev_df.sort_values("spk_id").groupby("spk_id").apply(lambda x: x.id.values)

with open("/dataset/SV_sets/voices/kaldi_files/spk2utt", "w") as f:
    for k, v in spk2utt.iteritems():
        line = ' '.join([k]+v.tolist())
        f.write(line+'\n')

### Dev Distribution

In [141]:
dev_df.groupby('spk_id').ch_id.value_counts()

spk_id  ch_id   
sp0032  ch021625    32
        ch021631    32
        ch021634    32
sp0083  ch009960    32
        ch011691    32
        ch003054     2
sp0093  ch126208    64
        ch126209    32
sp0112  ch121727    96
        ch123215    32
        ch123216    32
sp0122  ch121730    96
        ch129752    32
sp0150  ch126112    64
        ch132655    32
        ch126107     4
sp0159  ch121891     2
sp0174  ch084280    32
        ch050561     2
sp0188  ch136483    32
        ch141613    32
        ch135249     4
sp0196  ch122152    96
        ch122159    64
        ch122150     2
sp0198  ch126831    96
        ch000209     2
sp0204  ch287139    32
sp0205  ch159056    64
        ch123882     2
                    ..
sp7981  ch112057    32
        ch112058    32
sp7995  ch276908    64
        ch280250    32
sp8051  ch119902    32
        ch295385    32
        ch118101     2
sp8057  ch284428    32
        ch296395    32
sp8108  ch280354    32
        ch280359    32
sp8118  ch114476 

In [166]:
# enrollemend room("rm1") and test room("rm2")
dev_df.room.value_counts()

rm2    15648
rm1      256
Name: room, dtype: int64

In [167]:
dev_df.noise.value_counts()

none    4168
tele    3912
babb    3912
musi    3912
Name: noise, dtype: int64

In [168]:
dev_df.spk_id.value_counts()

sp5400    194
sp1160    192
sp0288    192
sp0949    192
sp6241    164
sp1392    164
sp0597    162
sp2156    162
sp0472    162
sp0196    162
sp7867    162
sp0636    162
sp6415    160
sp7498    160
sp5338    160
sp0112    160
sp6895    160
sp2532    160
sp1272    160
sp4967    132
sp3645    132
sp1963    132
sp7264    130
sp5189    130
sp1851    130
sp7247    130
sp7540    130
sp1867    130
sp2758    128
sp0770    128
         ... 
sp8635     34
sp3446     34
sp1737     34
sp1052     32
sp1335     32
sp3994     32
sp7276     32
sp1182     32
sp1383     32
sp4145     32
sp8575     32
sp2074     32
sp1970     32
sp1607     32
sp0204     32
sp5157     32
sp6519     32
sp1961     32
sp2012     32
sp0307     32
sp6544     32
sp7517     32
sp4586     32
sp8605      4
sp4331      2
sp4110      2
sp2412      2
sp1212      2
sp0403      2
sp0159      2
Name: spk_id, Length: 196, dtype: int64

### VOiECE trials

In [112]:
trial_list = pd.read_csv("/dataset/SV_sets/voices/original_data/sid_dev_lists_and_keys/dev-trial-keys.lst", delimiter=" ", names=["enroll_id", "test_id", "label"])
# trial_list.test_id = trial_list.test_id.apply(lambda x: x.split("/")[2].rstrip(".wav"))

In [121]:
dev_ids = dev_df.id.tolist()
id2idx = {v:i for i, v in enumerate(dev_ids)}
enr_idx = trial_list.enroll_id.apply(lambda x: id2idx[x])
test_idx = trial_list.test_id.apply(lambda x: id2idx[x])
trial_list['enroll_idx'] = enr_idx
trial_list['test_idx'] = test_idx
trial_list.loc[trial_list.label == 'imp', 'label'] = 0
trial_list.loc[trial_list.label == 'tgt', 'label'] = 1
trial_list.to_csv("/dataset/SV_sets/voices/voices_dev_trial.csv", index=False)

In [118]:
trial_list.label.value_counts()

0    3985792
1      20096
Name: label, dtype: int64

In [220]:
print(f"target_ratio: {20096 / 3985792}")

target_ratio: 0.005041908860271685


## Eval Set

무조건 id는 패턴을 맞춰주는게 좋다 speaker-id로 시작해야 sorting이 올바르게 된다.

In [299]:
eval_enroll = pd.read_csv("/dataset/SV_sets/voices/eval_set/sid_eval_lists/eval-enroll.lst", delimiter=" ", names=["id", "file"])
eval_test = pd.read_csv("/dataset/SV_sets/voices/eval_set/sid_eval_lists/eval-test.lst", delimiter=" ", names=["id"])
eval_test["file"] = eval_test.id
eval_test["id"] = eval_test.id.apply(lambda x: x.split("/")[1].rstrip(".wav"))
eval_df = pd.concat([eval_enroll, eval_test])
fake_spk_id = ["spk"+str(i).zfill(5) for i in range(len(eval_df))]
eval_df["spk_id"] = fake_spk_id
eval_df["uttr_id"] = eval_df.apply(lambda x: x.spk_id + "/" + x.id, axis=1)
eval_df["file"] = eval_df.file.apply(lambda x: x.rstrip(".wav"))
eval_df = eval_df.reset_index(drop=True)
eval_df.to_csv("/dataset/SV_sets/voices/eval_set/voices_eval.csv")

In [301]:
eval_df

,id,file,spk_id,uttr_id
0,VOiCES_2019_Challenge_SID_eval_0001,sid_eval/VOiCES_2019_Challenge_SID_eval_0001,spk00000,spk00000/VOiCES_2019_Challenge_SID_eval_0001
1,VOiCES_2019_Challenge_SID_eval_0002,sid_eval/VOiCES_2019_Challenge_SID_eval_0002,spk00001,spk00001/VOiCES_2019_Challenge_SID_eval_0002
2,VOiCES_2019_Challenge_SID_eval_0003,sid_eval/VOiCES_2019_Challenge_SID_eval_0003,spk00002,spk00002/VOiCES_2019_Challenge_SID_eval_0003
3,VOiCES_2019_Challenge_SID_eval_0004,sid_eval/VOiCES_2019_Challenge_SID_eval_0004,spk00003,spk00003/VOiCES_2019_Challenge_SID_eval_0004
4,VOiCES_2019_Challenge_SID_eval_0005,sid_eval/VOiCES_2019_Challenge_SID_eval_0005,spk00004,spk00004/VOiCES_2019_Challenge_SID_eval_0005
5,VOiCES_2019_Challenge_SID_eval_0006,sid_eval/VOiCES_2019_Challenge_SID_eval_0006,spk00005,spk00005/VOiCES_2019_Challenge_SID_eval_0006
6,VOiCES_2019_Challenge_SID_eval_0007,sid_eval/VOiCES_2019_Challenge_SID_eval_0007,spk00006,spk00006/VOiCES_2019_Challenge_SID_eval_0007
7,VOiCES_2019_Challenge_SID_eval_0008,sid_eval/VOiCES_2019_Challenge_SID_eval_0008,spk00007,spk00007/VOiCES_2019_Challenge_SID_eval_0008
8,VOiCES_2019_Challenge_SID_eval_0009,sid_eval/VOiCES_2019_Challenge_SID_eval_0009,spk00008,spk00008/VOiCES_2019_Challenge_SID_eval_0009
9,VOiCES_2019_Challenge_SID_eval_0010,sid_eval/VOiCES_2019_Challenge_SID_eval_0010,spk00009,spk00009/VOiCES_2019_Challenge_SID_eval_0010


In [302]:
wavs = eval_df.file
ids = eval_df.uttr_id
pd.DataFrame(ids).assign(wav=wavs).to_csv("/dataset/SV_sets/voices/eval_set/kaldi_files/wav.scp", index=False, sep=' ', header=None)

In [303]:
spk2utt = eval_df.sort_values("spk_id").groupby("spk_id").apply(lambda x: x.uttr_id.values)
with open("/dataset/SV_sets/voices/eval_set/kaldi_files/spk2utt", "w") as f:
    for k, v in spk2utt.iteritems():
        line = ' '.join([k]+v.tolist())
        f.write(line+'\n')

In [315]:
eval_trial = pd.read_csv("/dataset/SV_sets/voices/eval_set/sid_eval_lists/eval-trial.lst", delimiter=" ", names=["enroll_id", "test_id"])
eval_trial["test_id"] = eval_trial.test_id.apply(lambda x: x.split("/")[1].rstrip(".wav"))
# eval_ids = eval_df.id.tolist()
# id2idx = {v:i for i, v in enumerate(eval_ids)}
# enr_idx = eval_trial.enroll_id.apply(lambda x: id2idx[x])
# test_idx = eval_trial.test_id.apply(lambda x: id2idx[x])
# eval_trial['enroll_idx'] = enr_idx
# eval_trial['test_idx'] = test_idx

eval_trial.to_csv("/dataset/SV_sets/voices/voices_eval_trial.csv", index=False)
eval_trial[["enroll_id", "test_id"]].to_csv("/dataset/SV_sets/voices/eval_set/kaldi_files/voices_eval_sv", sep=' ', header=None, index=False)

## Join to VoxCeleb12

In [146]:
voxc12_si = pd.read_csv("/dataset/SV_sets/voxceleb12/dataframes/voxc12_si.csv")
voxc12_sv = pd.read_csv("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv.csv")

In [154]:
voxc12_dev = pd.concat([voxc12_si, voxc12_sv])
voxc12_dev = voxc12_dev[['id', 'spk', 'file']]

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [155]:
voices_dev = pd.read_csv("/dataset/SV_sets/voices/voices_dev.csv")

In [160]:
voices_dev = voices_dev[['id', 'spk_id', 'file']]
voices_dev.columns = 

In [159]:
voices_dev.columns

Index(['id', 'file', 'room', 'noise', 'spk_id', 'ch_id', 'sg_id', 'mc_id',
       'mc_t', 'mc_l', 'mc_deg', 'set'],
      dtype='object')